# Optimal weight table

In [1]:
import numpy as np
import networkx as nx
from itertools import product
import random
import matplotlib.pyplot as plt

In [2]:
length = 3
alphabet = ['U', 'C', 'A', 'G']
nodes = [''.join(p) for p in product(alphabet, repeat=length)]

In [3]:
SGC = [
    ["UUU", "UUC"],  # Phenylalanine (Phe)
    ["UUA", "UUG", "CUU", "CUC", "CUA", "CUG"],  # Leucine (Leu)
    ["AUU", "AUC", "AUA"],  # Isoleucine (Ile)
    ["AUG"],  # Methionine (Met) - Start codon
    ["GUU", "GUC", "GUA", "GUG"],  # Valine (Val)
    ["UCU", "UCC", "UCA", "UCG", "AGU", "AGC"],  # Serine (Ser)
    ["CCU", "CCC", "CCA", "CCG"],  # Proline (Pro)
    ["ACU", "ACC", "ACA", "ACG"],  # Threonine (Thr)
    ["GCU", "GCC", "GCA", "GCG"],  # Alanine (Ala)
    ["UAU", "UAC"],  # Tyrosine (Tyr)
    ["UAA", "UAG", "UGA"],  # Stop codons
    ["CAU", "CAC"],  # Histidine (His)
    ["CAA", "CAG"],  # Glutamine (Gln)
    ["AAU", "AAC"],  # Asparagine (Asn)
    ["AAA", "AAG"],  # Lysine (Lys)
    ["GAU", "GAC"],  # Aspartic acid (Asp)
    ["GAA", "GAG"],  # Glutamic acid (Glu)
    ["UGU", "UGC"],  # Cysteine (Cys)
    ["UGG"],  # Tryptophan (Trp)
    ["CGU", "CGC", "CGA", "CGG", "AGA", "AGG"],  # Arginine (Arg)
    ["GGU", "GGC", "GGA", "GGG"],  # Glycine (Gly)
]

In [18]:
swap_index =  {
    0: {#U
        0: 
        #C    #A    #G   
        {1: 0, 2: 1, 3: 2},
        #C
        1:
        #A    #G
        {2: 3, 3: 4},
        #A
        2:
        #G
        {3: 5}
        },
    1: {#U
        0: 
        #C    #A    #G   
        {1: 6, 2: 7, 3: 8},
        #C
        1:
        #A    #G
        {2: 9, 3: 10},
        #A
        2:
        #G
        {3: 11}
        },
    2: {#U
        0: 
        #C    #A    #G   
        {1: 12, 2: 13, 3: 14},
        #C
        1:
        #A    #G
        {2: 15, 3: 16},
        #A
        2:
        #G
        {3: 17}
        },
}

table_ones = np.ones(18)

table_altered = [1, 1, 1, 1, 1, 1,
                 1, 1, 1, 1, 1, 1,
                 4, 2, 2, 2, 2, 4]

In [5]:
def weights(s1, s2, table):
    if sum(a != b for a, b in zip(s1, s2)) == 1:
        if s1[0] != s2[0]:
            # base 1
            letter1 = s1[0]
            letter2 = s2[0]
            base = 0
        elif s1[1] != s2[1]:
            # base 2
            letter1 = s1[1]
            letter2 = s2[1]
            base = 1
        else:
            # base3
            letter1 = s1[2]
            letter2 = s2[2]
            base = 2
    
        l1 = ['U', 'C', 'A', 'G'].index(letter1)
        l2 = ['U', 'C', 'A', 'G'].index(letter2)
        if l1 > l2:
            l1, l2 = l2, l1
        return table[swap_index[base][l1][l2]]

In [6]:
def conductance(S, G):
    numerator = 0
    denominator = 0
    for edge, w in nx.get_edge_attributes(G, "weight").items():
        node1, node2 = edge
        if node1 in S and node2 in S:
            denominator += 2 * w
        else:
            numerator += w
            denominator += w

    return numerator / denominator

In [7]:
def mean_conductance(table):
    conductances = []
    for S in SGC:
        G = nx.Graph()
        for i, node1 in enumerate(S):
            for node2 in nodes:
                if node2 == node1:
                    continue
                w = weights(node1, node2, table)
                if w:
                    G.add_edge(node1, node2, weight=w)
                    
        conductances.append(conductance(S, G))

    return np.mean(conductances)

In [8]:
def fitness(table):
    return 1 - mean_conductance(table)

In [19]:
for table in table_ones, table_altered:
    print(fitness(table))

0.1887125220458553
0.3605442176870749


In [10]:
bias_factor = 1
pop_size = 10
probabilities = np.exp(-bias_factor * np.arange(pop_size) / pop_size)
probabilities /= np.sum(probabilities)
print(probabilities)
print(np.random.choice(pop_size, size=2, p=probabilities, replace=False))

[0.15054499 0.13621874 0.12325581 0.11152647 0.10091332 0.09131015
 0.08262084 0.07475843 0.06764422 0.06120702]
[3 8]


In [11]:
def initialize_population(pop_size, num_weights):
    return np.random.rand(pop_size, num_weights)

def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, 18)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def mutate(weights, mutation_rate=0.4):
    index = np.random.randint(18)
    factor = np.random.uniform(1 / (1 + mutation_rate), (1 + mutation_rate))
    weights[index] *= factor
    return weights

def evolutionary_algorithm(fitness, pop_size=150, crossover_rate=0.6, mutation_rate=0.4, delta=1e-6, max_generations=1000, bias_factor=1):
    population = initialize_population(pop_size, 18)
    best_fitness = 0
    generations = 0
    
    probabilities = np.exp(-bias_factor * np.arange(pop_size) / pop_size)
    probabilities /= np.sum(probabilities)
    
    while generations < max_generations:
        fitness_values = np.array([fitness(ind) for ind in population])
        sorted_indices = np.argsort(fitness_values)[::-1]
        population = population[sorted_indices]
        new_population = []
      
        for i in range(int(crossover_rate * pop_size / 2)):
            parent1_index, parent2_index = np.random.choice(pop_size, size=2, p=probabilities, replace=False)
            parent1, parent2 = population[parent1_index], population[parent2_index]
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([child1, child2])
        
        while len(new_population) < pop_size:
            new_population.append(population[np.random.choice(pop_size, p=probabilities)].copy())
        
        new_population = np.array([mutate(ind, mutation_rate) for ind in new_population])
        
        new_best_fitness = max(fitness_values)
        if new_best_fitness > best_fitness:
            best_fitness = new_best_fitness
        print(f"{best_fitness:.5f}, {new_best_fitness:.5f}", generations)
        
        population = new_population
        generations += 1
    
    fitness_values = np.array([fitness(ind) for ind in population])
    sorted_indices = np.argsort(fitness_values)[::-1]
    population = population[sorted_indices]
    best_individual = population[0]
    
    return best_individual / np.sum(best_individual) * 18  # Normalize weights

In [12]:
best_ind = evolutionary_algorithm(fitness, pop_size=40, max_generations=500)

0.28719, 0.28719 0
0.31735, 0.31735 1
0.33014, 0.33014 2
0.33014, 0.32895 3
0.36028, 0.36028 4
0.36028, 0.33127 5
0.36028, 0.33278 6
0.36028, 0.34114 7
0.36028, 0.35176 8
0.39139, 0.39139 9
0.40369, 0.40369 10
0.40369, 0.39017 11
0.40369, 0.39705 12
0.40369, 0.40244 13
0.40369, 0.40255 14
0.41024, 0.41024 15
0.41024, 0.40113 16
0.43183, 0.43183 17
0.43183, 0.43159 18
0.43946, 0.43946 19
0.44055, 0.44055 20
0.46558, 0.46558 21
0.46558, 0.46553 22
0.48551, 0.48551 23
0.52491, 0.52491 24
0.52491, 0.47970 25
0.52491, 0.48078 26
0.52491, 0.49229 27
0.52491, 0.48940 28
0.52491, 0.49412 29
0.52491, 0.49539 30
0.52491, 0.49913 31
0.52491, 0.52245 32
0.52955, 0.52955 33
0.55679, 0.55679 34
0.55996, 0.55996 35
0.56170, 0.56170 36
0.58724, 0.58724 37
0.58935, 0.58935 38
0.59827, 0.59827 39
0.59956, 0.59956 40
0.60118, 0.60118 41
0.60204, 0.60204 42
0.61640, 0.61640 43
0.62159, 0.62159 44
0.62408, 0.62408 45
0.62408, 0.62249 46
0.62408, 0.62403 47
0.62655, 0.62655 48
0.62972, 0.62972 49
0.63757, 0

In [13]:
np.set_printoptions(precision=10, suppress=True)
print(best_ind)

print(mean_conductance(best_ind))
print(fitness(best_ind))

[ 0.0000000582  0.0000000219  0.0000000501  0.0000000396  0.0000000174
  0.0000000588  0.0000000078  0.000000023   0.0000002379  0.0000000159
  0.0000000494  0.0000000806 17.9926964905  0.0000000488  0.0000000552
  0.0000000315  0.0000000737  0.0073026397]
0.11112905514537565
0.8888709448546244


In [14]:
from tabulate import tabulate

# Column headers
headers = ["", "U", "C", "A", "G"]  # Empty first header for row labels

alphabet = ["U", "C", "A", "G"]

data = []

for l1 in alphabet:
    #from 
    po1 = alphabet.index(l1)
    row = []
    for l2 in alphabet:
        #to
        if l1 == l2:
            row.append(0)
        else:
            po2 = alphabet.index(l2)
            if po1 > po2:
                tmp_po1, tmp_po2 = po2, po1
            else:
                tmp_po1, tmp_po2 = po1, po2
            row.append(best_ind[swap_index[2][tmp_po1][tmp_po2]])
            print(l1, l2, best_ind[swap_index[2][tmp_po1][tmp_po2]])
    data.append(row)

# Add row labels to data
data_with_labels = [[label] + list(row) for label, row in zip(alphabet, data)]

# Print table
print(tabulate(data_with_labels, headers=headers, tablefmt="grid"))

U C 17.992696490461903
U A 4.8834175022539625e-08
U G 5.5155852533900113e-08
C U 17.992696490461903
C A 3.1494335937304115e-08
C G 7.368221853842636e-08
A U 4.8834175022539625e-08
A C 3.1494335937304115e-08
A G 0.007302639724115181
G U 5.5155852533900113e-08
G C 7.368221853842636e-08
G A 0.007302639724115181
+----+--------------+--------------+-------------+-------------+
|    |            U |            C |           A |           G |
+====+==============+==============+=============+=============+
| U  |  0           | 17.9927      | 4.88342e-08 | 5.51559e-08 |
+----+--------------+--------------+-------------+-------------+
| C  | 17.9927      |  0           | 3.14943e-08 | 7.36822e-08 |
+----+--------------+--------------+-------------+-------------+
| A  |  4.88342e-08 |  3.14943e-08 | 0           | 0.00730264  |
+----+--------------+--------------+-------------+-------------+
| G  |  5.51559e-08 |  7.36822e-08 | 0.00730264  | 0           |
+----+--------------+--------------+----

In [15]:
def weight(s1, s2):
    s1 = list(s1)
    s2 = list(s2)
    diff = (np.array(s1) == np.array(s2)).astype(int)
    if sum(diff) == 2:
        if (s1[2] == "U" and s2[2] == "G") or (s1[2] == "G" and s2[2] == "U"):
            return 2
        if (s1[2] == "A" and s2[2] == "C") or (s1[2] == "C" and s2[2] == "A"):
            return 2
        if (s1[2] == "A" and s2[2] == "U") or (s1[2] == "U" and s2[2] == "A"):
            return 2
        if (s1[2] == "C" and s2[2] == "G") or (s1[2] == "G" and s2[2] == "C"):
            return 2
        
        if (s1[2] == "U" and s2[2] == "C") or (s1[2] == "C" and s2[2] == "U"):
            return 4
        if (s1[2] == "A" and s2[2] == "G") or (s1[2] == "G" and s2[2] == "A"):
            return 4
        else:
            return 1
    else:
        return False

In [16]:
def weight_popt(s1, s2):
    s1 = list(s1)
    s2 = list(s2)
    diff = (np.array(s1) == np.array(s2)).astype(int)
    if sum(diff) == 2:
        # position 1
        if (s1[0] == "U" and s2[0] == "G") or (s1[0] == "G" and s2[0] == "U"):
            return 0.003
        if (s1[0] == "A" and s2[0] == "C") or (s1[0] == "C" and s2[0] == "A"):
            return 0.003
        if (s1[0] == "A" and s2[0] == "U") or (s1[0] == "U" and s2[0] == "A"):
            return 0.002
        if (s1[0] == "C" and s2[0] == "G") or (s1[0] == "G" and s2[0] == "C"):
            return 0.003
        if (s1[0] == "U" and s2[0] == "C") or (s1[0] == "C" and s2[0] == "U"):
            return 0.006
        if (s1[0] == "A" and s2[0] == "G") or (s1[0] == "G" and s2[0] == "A"):
            return 0.005
        
        # position 2
        if (s1[1] == "U" and s2[1] == "G") or (s1[1] == "G" and s2[1] == "U"):
            return 0.003
        if (s1[1] == "A" and s2[1] == "C") or (s1[1] == "C" and s2[1] == "A"):
            return 0.003
        if (s1[1] == "A" and s2[1] == "U") or (s1[1] == "U" and s2[1] == "A"):
            return 0.002
        if (s1[1] == "C" and s2[1] == "G") or (s1[1] == "G" and s2[1] == "C"):
            return 0.004
        if (s1[1] == "U" and s2[1] == "C") or (s1[1] == "C" and s2[1] == "U"):
            return 0.002
        if (s1[1] == "A" and s2[1] == "G") or (s1[1] == "G" and s2[1] == "A"):
            return 0.005
        
        if (s1[2] == "U" and s2[2] == "G") or (s1[2] == "G" and s2[2] == "U"):
            return 0.007
        if (s1[2] == "A" and s2[2] == "C") or (s1[2] == "C" and s2[2] == "A"):
            return 0.021
        if (s1[2] == "A" and s2[2] == "U") or (s1[2] == "U" and s2[2] == "A"):
            return 0.018
        if (s1[2] == "C" and s2[2] == "G") or (s1[2] == "G" and s2[2] == "C"):
            return 0.012
        if (s1[2] == "U" and s2[2] == "C") or (s1[2] == "C" and s2[2] == "U"):
            return 15.925
        if (s1[2] == "A" and s2[2] == "G") or (s1[2] == "G" and s2[2] == "A"):
            return 1.977

    else:
        return False